In [1]:
import tensorflow as tf
from tensorflow.keras import *
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
model=Sequential()
model.add(layers.Dense(2,activation='softmax'))
model.compile(optimizer=optimizers.Adam(),
             # loss=keras.losses.CategoricalCrossentropy(),  # 需要使用to_categorical
             loss=losses.binary_crossentropy,
              metrics=['accuracy'])

In [3]:
data_path='VeReMi-Dataset/000_Select_data'
os.chdir(data_path)
datalist=os.listdir()
test=np.load('test.npy',allow_pickle=True)
val=np.load('val.npy',allow_pickle=True)
train=np.load('train.npy',allow_pickle=True)
train=train.T
val=val.T
test=test.T

In [4]:
def get_info(bucket_list,data_label_df):
    result=[]
    for i in data_label_df:
        result.append(i[bucket_list])
    return result

def regularize(raw_data):
    for df in raw_data:
        newDataFrame = pd.DataFrame(index=df.index)
        columns = df.columns.tolist()
        for c in columns:
            if c != 'type':
                d = df[c]
                try:
                    len(d[0])
                    tlist = []
                    for i in d:
                        tlist.append([])
                    for i in range(len(d[0])):
                        MAX = d[0][i]
                        for row in d:
                            if row[i] > MAX:
                                MAX = row[i]
                        MIN = d[0][i]
                        for row in d:
                            if row[i] < MIN:
                                MIN = row[i]
                        for j in range(len(d)):
                            if MAX == MIN:
                                tlist[j].append(1)
                            else:
                                tlist[j].append((d[j][i]-MIN)/ (MAX - MIN))
                    newDataFrame[c] = tlist
                except TypeError:
                    MAX = d.max()
                    MIN = d.min()
                    newDataFrame[c] = ((d - MIN) / (MAX - MIN)).tolist()
    return newDataFrame

In [16]:
loc=0
for x in train[0]:
    x.fillna(0,inplace=True)
    train[0][loc]=regularize(x)
    loc+=1
for x in test[0]:
    x.fillna(0,inplace=True)
for x in val[0]:
    x.fillna(0,inplace=True)

ValueError: operands could not be broadcast together with shapes (773,) (3,) 

In [11]:
np.asarray(train[1]).astype('float64')
np.asarray(val[1]).astype('float64')

history = model.fit(train[0], train[1], batch_size=64, epochs=10, validation_data=val,validation_split=0.1)

ValueError: setting an array element with a sequence.